In [51]:
import numpy as np
import pandas as pd
import re
import requests
import lxml
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time


In [52]:
sex = '男歌手'
df_singers = pd.read_csv(rf'./{sex}.csv')
df_singers

,歌手,链接
0,林俊杰,https://music.163.com/artist?id=3684
1,陈奕迅,https://music.163.com/artist?id=2116
2,郑润泽,https://music.163.com/artist?id=29051613
3,颜人中,https://music.163.com/artist?id=31376161
4,汪苏泷,https://music.163.com/artist?id=5538
...,...,...
95,THOME,https://music.163.com/artist?id=12110173
96,刘思达LOFTHESTAR,https://music.163.com/artist?id=12030367
97,杨和苏KeyNG,https://music.163.com/artist?id=13899303
98,MC HotDog 热狗,https://music.163.com/artist?id=4479


In [ ]:
# dict_songs = dict()
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:140.0) Gecko/20100101 Firefox/140.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "utf-8",
    "Connection": "keep-alive",
    "Referer": "https://music.163.com/",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "iframe",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Priority": "u=4",
    "TE": "trailers"
}
cookies = {
    '__csrf'         :  "e69faeccd93a14d89ccd4adaed646804",
    '__snaker__id'	 :  "2mTQGljb7kvL842G",
    '_iuqxldmzr_'	 :  "32",
    '_ntes_nnid'	 :  "fafe5ac96e2d4ae498fca20fb41d0dc8,1750860195486",
    '_ntes_nuid'	 :  "fafe5ac96e2d4ae498fca20fb41d0dc8",
    'csrfToken'	     :  "ZfVV9cRIAgXWdd56noaXflY5",
    'gdxidpyhxdE'    :  "hdVn9iYxQt1EMBRLipauqgUfVzM6lPgrSSoG\\+zGqsAwfCAGW39rntkGIwURsvnva9dtvT1PmnCL0ZdWe6BZAIbp2/vQXwWLKXHOUaEpQ/kK8e3Oix0ahXu/M8NR+wlg1fLm1dgtjwcbJ+1glTs92oYxfqZjluGpHymRHPKqE2zzySO+:1751019940224",
    'JSESSIONID-WYYY':	"2+\\uNX25zzTYHlkjgvte3U27lslNxjixte4Gh\\C\\5x3G6SqQCfmGENW3Xg9gjzipo/s0ukj5v2QT88DDdP9opdk2C42SwXfMd1dTyOcs64U/X9zGQPFOr8mryUCQDoPQ5VSv/E3c9zabfkFSY4Yjscr97rWyMjGdkrdIqd8gg2a70yMM:1751019798915",
    'MUSIC_U'	     :  "0033B13615A496B22EB00B5690DBD70B1C27639A95C33DEACE4F002C5823958B4AD6E80C1E592F52622EF7C610D46FC0EE525B07ED1C1C2E8C53DA638256F623C34D96B308D77D2AE71EA670B6BE86C6B59B0B90BA1F7EA95B43AB07262247AA6768BBC881D3A2515829DAB90CDFA912AD39E243329B06F6FA64C5FE0662EBE4AABB271CA287A8F65A443E8B7F3CD9341059EA0A22A2C7601145605258AADE11F81C6DD9269AAE7065995D376DC350CB6EC128F06E262C117D1AD7AA18B31CDADCF4053789CC4B41FD69351188312281F9B7487559FC5ACB4BFE40E977345AB584E2D1BFFC33C0CD3BDB1C4B829A0F79DF01F3E4D6E0BD4EDDA491CD337FC3C3713BC912CE280DEFB1BCFA7A857E4B3A9243792B70A51DE516765201DEF46B15D1CD34EA4BA7A62B8A1C99F5409950212F",
    'NMTID'	         :  "00ORRrO9Mr7TBwzukUvj96KI48ymJUAAAGXp2Zncg",
    'ntes_kaola_ad'	 :  "1",
    'ntes_utid'	     :  "tid._.%2BChCgTThg5lFUgARFRPTbsk%2FKGh6yRzI._.0",
    'sDeviceId'	     :  "YD-nekKbrguHYNEB0AQFEKCask+PDl/2Fyd",
    'WEVNSM'	     :  "1.0.0",
    'WM_NI'	         :  "Zy2YxEL2rvKOjuOuOQgVBpqV76Yt8FtEcS6pSTBSPDqYSl4lzTmO+Htb+b/xxv2gIgWtqXIWyd7iOFG0Exl1JLE6G/KgbpHCtznBIJ2g0ZCNqFH//cv1BuKp/hVyzq9rWXo=",
    'WM_NIKE'	     :  "9ca17ae2e6ffcda170e2e6eea2f97e9a958393db4e878e8fa3d85b879f9b86d26ef7a6a88ee1339abb8c84d02af0fea7c3b92a9aa7febafb4f90eebe97ca45aaab8196ce5ba58a8e91f45ef5ea9795bb4eb89badb4b57c918c8eb4d350b3879d8db768ba8ca3b0aa44b8bee583ae74bcaafe88cc6d95acbfb9cc5dbbbcf98ace4e85f08b99ef3b8a948bd7c62589b9ae91cb49b8a6a2a3cc42f2b900bbc246b39a88d0ae69b6ef8e96ca7cedb89a9af97bfbae99b9ee37e2a3",
    'WM_TID'	     :  "KA7NcHQUjuRFVERBRVPWL5x/LH0/T9Dy",
    'WNMCID'	     :  "aomccp.1750860197302.01.0",
}

df_songs = pd.DataFrame({
    '歌手': [],
    '歌曲': [],
    '歌曲链接': []
})
tqdm.pandas()

def get_songs(row):
    response = requests.get(url=row.链接, headers=headers, params={}, cookies=cookies)
    soup = bs(response.text, 'lxml')
    # print(soup.prettify())
    # pbar = tqdm.get_lock().get_instances()[0]
    # pbar.set_description_str(f'processing {row.歌手}')

    song_names = [song.text for song in soup.find_all('a', href=re.compile(r'/song\?id=\d'))]
    song_links = [r'https://music.163.com' + song['href'] for song in soup.find_all('a', href=re.compile(r'/song\?id=\d'))]

    global df_songs
    df_songs = pd.concat(
        [
            df_songs,
            pd.DataFrame({
                '歌手': [row.歌手 for _ in range(len(song_names))],
                '歌曲': song_names,
                '歌曲链接': song_links
            })
        ],
        ignore_index=True
    ) 
    # return df_songs_new
    # time.sleep(2.0)

    

In [54]:
df_singers.progress_apply(get_songs, axis=1)

100%|██████████| 100/100 [01:04<00:00,  1.54it/s]


0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Length: 100, dtype: object

In [55]:
df_songs.to_csv(rf'./{sex}歌曲.csv', header=True, index=False)